In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn import preprocessing , neighbors ,svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from __future__ import absolute_import, division, print_function, unicode_literals


try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Version:  2.15.0
Eager mode:  True
Hub version:  0.16.1
GPU is available


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def fun_remove_stop_words(posts):

    filtered=''

    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x

    return filtered

In [ ]:
plain_file_path = '/content/drive/MyDrive/plain.txt'

# Read the text file into a Pandas DataFrame with 'Query' column
plain= pd.read_csv(plain_file_path, header=None, names=['Query'], delimiter='\t', engine='python')
plain = plain.iloc[:-22, :]
print(len(plain))
plain_text = plain['Query'].values
data=''
for x in plain_text:
    data+=" " + x
data=fun_remove_stop_words(data)  # remove stop words
data=data.split('.')
plain['Query']= plain_text
# Add empty columns for 'Length', 'Attack', and 'Label'
plain['Length'] = '0'
plain['Attack'] = '0'
plain['Label'] = ''

3672


In [ ]:
import pandas as pd
sql = pd.read_csv('/content/drive/MyDrive/sql.csv')
password = pd.read_csv('/content/drive/MyDrive/password.csv')
username = pd.read_csv('/content/drive/MyDrive/username.csv')
sqli = pd.read_csv('/content/drive/MyDrive/sqli.csv')

In [ ]:
sqli



,Query,Length,Attack,Label
0,1' where 6406=6406;select count(*) from rdb$fi...,115,sqli,anom
1,1) and 8514=(select count(*) from domain.domai...,111,sqli,anom
2,-3136%') or 3400=6002,21,sqli,anom
3,1) where 7956=7956 or sleep(5)#,31,sqli,anom
4,-7387'))) order by 1--,22,sqli,anom
...,...,...,...,...
10846,-7773' or 5903=('qqpjq'||(select case 5903 whe...,99,sqli,anom
10847,"1') and extractvalue(7982,concat(0x5c,0x717170...",111,sqli,anom
10848,"1"" order by 1--",15,sqli,anom
10849,"1' procedure analyse(extractvalue(5840,concat(...",149,sqli,anom


In [ ]:


username.dropna(axis=0,how="all",inplace=True)
password.dropna(axis=0,how="all",inplace=True)
username.reset_index(drop = True,inplace=True)
password.reset_index(drop = True,inplace=True)


In [ ]:
def leng(df,col,len_col):
  for i in range(len(df)):
    cl = df[col][i]
    length = len(str(cl))
    df[len_col][i] = length
  return df

In [ ]:
username = leng(username, 'Query','Length')
password = leng(password, 'Query','Length')
sql = leng(sql, 'Query','Length')
plain = leng(plain,'Query','Length')

<ipython-input-113-c4d3d729a6ae>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[len_col][i] = length


In [ ]:
sqli['Label'] = 'sqli'
username['Label'] = 'username'
password['Label'] = 'password'
sql['Label'] = 'sql'
plain['Label']='plain'

In [ ]:
username.drop(['Attack'],axis=1,inplace=True)
password.drop(['Attack'],axis=1,inplace=True)
sql.drop(['Attack'],axis=1,inplace=True)
sqli.drop(['Attack'],axis=1,inplace=True)
plain.drop(['Attack'],axis=1,inplace=True)


In [ ]:

df = pd.concat([sqli,sql,username,password,plain])
df.reset_index(drop=True,inplace=True)
mask = df['Query'].notna()
df = df[mask]
df


,Query,Length,Label
0,1' where 6406=6406;select count(*) from rdb$fi...,115,sqli
1,1) and 8514=(select count(*) from domain.domai...,111,sqli
2,-3136%') or 3400=6002,21,sqli
3,1) where 7956=7956 or sleep(5)#,31,sqli
4,-7387'))) order by 1--,22,sqli
...,...,...,...
16554,and1,4,plain
16555,ap14,4,plain
16556,application,11,plain
16557,apply2,6,plain


In [ ]:
def cal_puncndop(df,col,punop_col,l):
  df1 = df[[col]].copy()
  for i, query in enumerate(df[col]):
    count = 0
    li = list(query)
    for ch in range(len(query)):
      if query[ch] in l:
        li [ch] = " "
        #df[col][i] = "".join(li)
        count = count+1
    df1[col][i] = "".join(li)
    #print(("".join(li)))
    df[punop_col][i] = count
  df[col] = df1[col]
  return df

In [ ]:
df['punctuation'] = 0
# df['operator'] = 0
df=cal_puncndop(df,'Query','punctuation',['!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?","[","]",")","("])
# df=cal_puncndop(df,'Query','operator',["<",">", "<=", ">=", "=", "==", "!=", "<<", ">>", "|", "&", "-", "+", "%", "^", "*"]) #should * be included becoz count(*) !!!
df.head()

<ipython-input-129-4359141fa350>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[punop_col][i] = count


,Query,Length,Label,punctuation
0,1 where 6406=6406 select count * from rdb$fi...,115,1,0
1,1 and 8514= select count * from domain domai...,111,1,0
2,3136% or 3400=6002,21,1,0
3,1 where 7956=7956 or sleep 5 #,31,1,0
4,7387 order by 1,22,1,0


In [ ]:
def cal_keyword(df,col,key_col,l):
  for i, query in enumerate(df[col]):
    count = 0
    query = query.lower()
    words = query.split()
    for word in words:
      if word in l:
        count = count+1
    df[key_col][i] = count
  return df

In [ ]:
df['keyword'] = 0
df = cal_keyword(df, 'Query', 'keyword', ["select", "update", "insert", "create", "drop", "alter", "rename", "exec", "order", "group", "sleep","count","where"])
df.head()

<ipython-input-131-85b86fb67671>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key_col][i] = count


,Query,Length,Label,punctuation,keyword
0,1 where 6406=6406 select count * from rdb$fi...,115,1,0,3
1,1 and 8514= select count * from domain domai...,111,1,0,2
2,3136% or 3400=6002,21,1,0,0
3,1 where 7956=7956 or sleep 5 #,31,1,0,2
4,7387 order by 1,22,1,0,1


In [ ]:
for i,label in enumerate(df['Label']):
  if label in ['sql','username','password','plain']:
    df['Label'][i] = 'non-sqli'


In [ ]:

def encode_categorical(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('str')
        encoder = preprocessing.LabelEncoder()
        encoded_list = encoder.fit_transform(df[column])
        # print(encoded_list)
#         print(len(encoded_list))
        encoded_series = pd.Series(encoded_list)
        df[column] = encoded_series
        print("The ", column, "is encoded ")
    return(df)

df = encode_categorical(df,['Label'])



The  Label is encoded 


In [ ]:
X = np.array(df.drop(labels=['Label', 'Query'],axis=1)).reshape(len(df),1,3)
y = np.array(df['Label'])



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = tf.constant(X_train, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.float32)
X_test = tf.constant(X_test, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.float32)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset1 = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset1 = test_dataset.batch(BATCH_SIZE)


In [ ]:


# # Define batch size
# BATCH_SIZE = 64

# # Calculate the size of the last batch
# last_batch_size = len(X_test) % BATCH_SIZE

# # Pad only the last batch if it's not already equal to BATCH_SIZE
# if last_batch_size > 0:
#     # Extract the last slice
#     last_slice = test_dataset.take(last_batch_size)

#     # Pad the last slice with existing values (not along the sequence dimension)
#     padded_last_slice = last_slice.padded_batch(BATCH_SIZE, padded_shapes=([None, 3], []))  # Adjust padded_shapes based on your data structure

#     # Combine the padded last slice with the rest of the dataset
#     test_dataset_padded = test_dataset.skip(last_batch_size).concatenate(padded_last_slice)
# else:
#     # If the last batch size is already equal to BATCH_SIZE, use the original dataset
#     test_dataset_padded = test_dataset

# # Batch the final padded dataset
# test_dataset_padded = test_dataset_padded.batch(BATCH_SIZE)


In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, activation='relu', input_shape=(1, 3)))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Flatten())  # Flatten the output
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1, 20)             80        
                                                                 
 dense_4 (Dense)             (None, 1, 16)             336       
                                                                 
 flatten_1 (Flatten)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 433 (1.69 KB)
Trainable params: 433 (1.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
for batch in train_dataset1.take(1):
    inputs, labels = batch
    print("Input Shape:", inputs.shape)
    print("Label Shape:", labels.shape)

Input Shape: (64, 1, 3)
Label Shape: (64,)


In [ ]:
history = model.fit(train_dataset1,
                    epochs=20,
                    verbose=1)

Epoch 1/20
207/207 [==============================] - 2s 3ms/step - loss: 0.9708 - accuracy: 0.7504
Epoch 2/20
207/207 [==============================] - 1s 3ms/step - loss: 0.4876 - accuracy: 0.8102
Epoch 3/20
207/207 [==============================] - 1s 4ms/step - loss: 0.4211 - accuracy: 0.8361
Epoch 4/20
207/207 [==============================] - 1s 3ms/step - loss: 0.3809 - accuracy: 0.8428
Epoch 5/20
207/207 [==============================] - 1s 4ms/step - loss: 0.3490 - accuracy: 0.8576
Epoch 6/20
207/207 [==============================] - 1s 5ms/step - loss: 0.3319 - accuracy: 0.8647
Epoch 7/20
207/207 [==============================] - 2s 8ms/step - loss: 0.3176 - accuracy: 0.8687
Epoch 8/20
207/207 [==============================] - 1s 3ms/step - loss: 0.3176 - accuracy: 0.8653
Epoch 9/20
207/207 [==============================] - 1s 3ms/step - loss: 0.3135 - accuracy: 0.8662
Epoch 10/20
207/207 [==============================] - 1s 3ms/step - loss: 0.3114 - accuracy: 0.8677

In [ ]:
model.evaluate(test_dataset1)

52/52 [==============================] - 0s 3ms/step - loss: 0.3137 - accuracy: 0.8699


[0.31368595361709595, 0.8698671460151672]

In [ ]:
model.save('/content/drive/MyDrive/sqlimodel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
sql_injection_keywords = [
    'select',
    'update',
    'insert',
    'create',
    'drop',
    'alter',
    'rename',
    'exec',
    'order',
    'group',
    'sleep',
    'count',
    'where'
]

In [ ]:
punc_list = ['!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?","[","]",")","("]

In [ ]:

def remove_and_count_specific_punctuation(input_string, punctuations_to_remove):
    # Create a translation table to map each specified punctuation character to None
    translator = str.maketrans('', '', ''.join(punctuations_to_remove))

    # Use translate method to remove specified punctuations from the input string
    no_specified_punctuations = input_string.translate(translator)

    # Count the occurrences of each specified punctuation character
    specified_punctuation_counts = len(input_string)-len(no_specified_punctuations)

    return specified_punctuation_counts

In [ ]:
def preprocess_input(user_input,keywords):
  length = len(str(user_input))
  keyword_count = sum(user_input.lower().count(keyword.lower()) for keyword in keywords)
  punc_count=remove_and_count_specific_punctuation(user_input,punc_list)
  df =pd.DataFrame({'Length':[length],'punctuation':[punc_count],'keyword':[keyword_count]},index=None)
  return df


In [ ]:
# user_input = 'select * from users where id = 1 or "{;" or 1 = 1 -- 1'
user_input = 'Update  from users where id=99 or "{;" or 1 = 1 -- 1 '
df=preprocess_input(user_input,sql_injection_keywords)

df = np.array(df).reshape(len(df),1,3)
tensor_df = tf.data.Dataset.from_tensor_slices(df).batch(64)

output=model.predict(tensor_df)
value=output[0,0]
print(value)

1/1 [==============================] - 0s 111ms/step
0.99995816
